In [28]:
import fine_tunning
import json

fine_tuning_data = []
stratify_array = []
for item in fine_tunning.examples:
    stratify_array.append("'question_type_code': '" + item["output"]["question_type_code"] + "'")
    fine_tuning_data.append({
        "messages": [
            {"role": "user", "content": item['input']},
            {"role": "assistant", "content": json.dumps(item['output'], ensure_ascii=False)},
        ]
    })
    
print(stratify_array)

["'question_type_code': 'a'", "'question_type_code': 'a'", "'question_type_code': 'a'", "'question_type_code': 'b'", "'question_type_code': 'b'", "'question_type_code': 'b'", "'question_type_code': 'b'", "'question_type_code': 'c'", "'question_type_code': 'c'", "'question_type_code': 'c'", "'question_type_code': 'c'", "'question_type_code': 'd'", "'question_type_code': 'd'", "'question_type_code': 'd'", "'question_type_code': 'd'", "'question_type_code': 'e'", "'question_type_code': 'e'", "'question_type_code': 'e'", "'question_type_code': 'e'", "'question_type_code': 'f'", "'question_type_code': 'f'", "'question_type_code': 'f'", "'question_type_code': 'f'", "'question_type_code': 'g'", "'question_type_code': 'g'", "'question_type_code': 'g'", "'question_type_code': 'g'", "'question_type_code': 'h'", "'question_type_code': 'h'", "'question_type_code': 'h'", "'question_type_code': 'h'", "'question_type_code': 'i'", "'question_type_code': 'i'", "'question_type_code': 'i'", "'question_ty

In [29]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    fine_tuning_data,
    test_size=0.25,
    stratify=stratify_array,
    random_state=42,
)

print(train_data)
print(val_data)

[{'messages': [{'role': 'user', 'content': '种西兰花时要提防哪些病'}, {'role': 'assistant', 'content': '{"crop": ["西兰花"], "pathogen": [], "soil": [], "growthStage": [], "question_type_code": "f"}'}]}, {'messages': [{'role': 'user', 'content': '锈菌容易危害哪些作物?'}, {'role': 'assistant', 'content': '{"crop": [], "pathogen": ["锈菌"], "soil": [], "growthStage": [], "question_type_code": "h"}'}]}, {'messages': [{'role': 'user', 'content': '种植豌豆时最重要的阶段是什么'}, {'role': 'assistant', 'content': '{"crop": ["豌豆"], "pathogen": [], "soil": [], "growthStage": [], "question_type_code": "e"}'}]}, {'messages': [{'role': 'user', 'content': '杏仁爱感染青枯病菌吗'}, {'role': 'assistant', 'content': '{"crop": ["杏仁"], "pathogen": ["青枯病菌"], "soil": [], "growthStage": [], "question_type_code": "m"}'}]}, {'messages': [{'role': 'user', 'content': '水稻适合在什么温度下生长'}, {'role': 'assistant', 'content': '{"crop": ["水稻"], "pathogen": [], "soil": [], "growthStage": [], "question_type_code": "b"}'}]}, {'messages': [{'role': 'user', 'content': '授粉和受精阶

In [30]:
def write_to_jsonl(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            json.dump(entry, f)
            f.write("\n")
            
            
training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [31]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

training_file = client.files.create(
    file=open(training_file_name, "rb"),purpose="fine-tune",
)

validation_file = client.files.create(
    file=open(validation_file_name, "rb"),purpose="fine-tune",
)

print("Training file id: " + training_file.id)
print("Validation file id: " + validation_file.id)

Training file id: file-YzuujHLwXQdxDA1HP47I2jUw
Validation file id: file-gJvwYUmUfhIXggWC4qO8FqJ6


In [32]:
suffix_name = "graduation_dingyi"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

response
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-XL835m6xrDXZbwqXlyTW2Psc', created_at=1714800697, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-im0rsa7K8uZbOeVhnVIswllJ', result_files=[], seed=1540444486, status='validating_files', trained_tokens=None, training_file='file-YzuujHLwXQdxDA1HP47I2jUw', validation_file='file-gJvwYUmUfhIXggWC4qO8FqJ6', estimated_finish=None, integrations=[], user_provided_suffix='graduation_dingyi'), FineTuningJob(id='ftjob-G384HyJ8O3tPTW1mZkZMmdjF', created_at=1714799543, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Unexpected file format, expected either prompt/completion pairs or chat messages.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparamet

In [33]:
import os

response = client.fine_tuning.jobs.retrieve(os.getenv("FINE_TUNE_ID"))
response

FineTuningJob(id='ftjob-XL835m6xrDXZbwqXlyTW2Psc', created_at=1714800697, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-im0rsa7K8uZbOeVhnVIswllJ', result_files=[], seed=1540444486, status='running', trained_tokens=None, training_file='file-YzuujHLwXQdxDA1HP47I2jUw', validation_file='file-gJvwYUmUfhIXggWC4qO8FqJ6', estimated_finish=None, integrations=[], user_provided_suffix='graduation_dingyi')

In [ ]:
def predict(test_query, fine_tune_model_id):
    messages = [{
        "role": "user",
        "content": test_query
    }]
    
    output = client.chat.completions.create(
        model = fine_tune_model_id, messages=messages, temperature=0, max_tokens=100
    )
    
    return output.choices[0].message.content